(sec:mongolite)=
# Requêtes depuis R : `mongolite`

* Auteurs/trices : **TODO**

Comment faire pour effectuer les requêtes présentes dans les chapitres 1 à 6 avec `mongolite` ?

Le fichier que vous devez modifier pour ce chapitre est `mongo_book/content/07_scripts_R.md`.

In [1]:
library(mongolite)
coll <- mongo("NYfood", url = "mongodb://localhost:27017/food")
print(coll)

<Mongo collection> 'NYfood' 
 $aggregate(pipeline = "{}", options = "{\"allowDiskUse\":true}", handler = NULL, pagesize = 1000, iterate = FALSE) 
 $count(query = "{}") 
 $disconnect(gc = TRUE) 
 $distinct(key, query = "{}") 
 $drop() 
 $export(con = stdout(), bson = FALSE, query = "{}", fields = "{}", sort = "{\"_id\":1}") 
 $find(query = "{}", fields = "{\"_id\":0}", sort = "{}", skip = 0, limit = 0, handler = NULL, pagesize = 1000) 
 $import(con, bson = FALSE) 
 $index(add = NULL, remove = NULL) 
 $info() 
 $insert(data, pagesize = 1000, stop_on_error = TRUE, ...) 
 $iterate(query = "{}", fields = "{\"_id\":0}", sort = "{}", skip = 0, limit = 0) 
 $mapreduce(map, reduce, query = "{}", sort = "{}", limit = 0, out = NULL, scope = NULL) 
 $remove(query, just_one = FALSE) 
 $rename(name, db = NULL) 
 $replace(query, update = "{}", upsert = FALSE) 
 $run(command = "{\"ping\": 1}", simplify = TRUE) 
 $update(query, update = "{\"$set\":{}}", filters = NULL, upsert = FALSE, multiple = FALSE)